In [1]:
from graph_generation.generate_graph import generate_erdos_renyi_graph, generate_erdos_renyi_attribute_graph, generate_barabasi_albert_graph, generate_barabasi_albert_attribute_graph
from diffusion_models.edge_probabilities import generate_edge_probabilities
from diffusion_models.independent_cascade import optimized_independent_cascade, dmp_ic, ALE_heuristic, modified_ALE, ALE_heuristic_transpose, IC_approx_vectorized, IC_approx_vectorized_torch
from influence_maximization.im import im_ic, im_diff
from training.train_edge_gnn import train_diffusion_gnn, train_edge_model_with_diffusion, train_edge_model
import numpy as np
import networkx as nx
import torch
from torch_geometric.utils import from_networkx
from torch_geometric.data import Data
from models.gnn_model import LearnableDiffusionGNN, MultiplicativeDiffusionGNN, ALE, ModifiedALE, ICApproxLayer, InfluenceSpreadNN, ICApproxLossModule
from plotting_tools.prob_plots import posterior_plots, edge_plots
import torch.nn.functional as F
from scipy.stats import spearmanr
from sklearn.metrics import roc_curve, roc_auc_score
from scipy import stats
import time
import random
size_x=10
size_y=3.5
seed = 42

random.seed(seed)                  
np.random.seed(seed)              
torch.manual_seed(seed)          
torch.cuda.manual_seed(seed)     
torch.cuda.manual_seed_all(seed)  
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [2]:
def summarise_stats(rmse, pearson, spearman, auc, time_ic_apporx, time_mc):
    ci_low, ci_high = stats.t.interval(0.95, df=len(rmse)-1, loc=rmse.mean(), scale=stats.sem(rmse))
    print(f"rmse:\n mean={rmse.mean()},\n ci_low={ci_low},\n ci_high={ci_high},\n std={rmse.std()}")

    ci_low, ci_high = stats.t.interval(0.95, df=len(pearson)-1, loc=pearson.mean(), scale=stats.sem(pearson))
    print(f"pearson:\n mean={pearson.mean()},\n ci_low={ci_low},\n ci_high={ci_high},\n std={pearson.std()}")

    ci_low, ci_high = stats.t.interval(0.95, df=len(spearman)-1, loc=spearman.mean(), scale=stats.sem(spearman))
    print(f"spearman:\n mean={spearman.mean()},\n ci_low={ci_low},\n ci_high={ci_high},\n std={spearman.std()}")

    ci_low, ci_high = stats.t.interval(0.95, df=len(auc)-1, loc=auc.mean(), scale=stats.sem(auc))
    print(f"auc:\n mean={auc.mean()},\n ci_low={ci_low},\n ci_high={ci_high},\n std={auc.std()}")

    ci_low, ci_high = stats.t.interval(0.95, df=len(time_ic_apporx)-1, loc=time_ic_apporx.mean(), scale=stats.sem(time_ic_apporx))
    print(f"time_ic_apporx:\n mean={time_ic_apporx.mean()},\n ci_low={ci_low},\n ci_high={ci_high},\n std={time_ic_apporx.std()}")

    ci_low, ci_high = stats.t.interval(0.95, df=len(time_mc)-1, loc=time_mc.mean(), scale=stats.sem(time_mc))
    print(f"time_mc:\n mean={time_mc.mean()},\n ci_low={ci_low},\n ci_high={ci_high},\n std={time_mc.std()}")

In [10]:
def compute_rmse(scalar, G, edge_dict, prior_probs, T, true_posterior_tensor):
    preds = IC_approx_vectorized(G, edge_dict, prior_probs, T, scalar)
    preds_tensor = torch.tensor(preds, dtype=torch.float32).unsqueeze(1)
    rmse = torch.nn.MSELoss()(preds_tensor, true_posterior_tensor).item() ** 0.5
    return rmse

def binary_search_optimal_scalar(G, edge_dict, prior_probs, T, true_posterior_tensor,
                                  low=0, high=10, tolerance=1e-7, max_iter=30):
    best_scalar = None
    best_rmse = float('inf')
    
    for _ in range(max_iter):
        mid1 = low + (high - low) / 3
        mid2 = high - (high - low) / 3

        rmse1 = compute_rmse(mid1, G, edge_dict, prior_probs, T, true_posterior_tensor)
        rmse2 = compute_rmse(mid2, G, edge_dict, prior_probs, T, true_posterior_tensor)

        if rmse1 < rmse2:
            high = mid2
            if rmse1 < best_rmse:
                best_rmse = rmse1
                best_scalar = mid1
        else:
            low = mid1
            if rmse2 < best_rmse:
                best_rmse = rmse2
                best_scalar = mid2

        if high - low < tolerance:
            break

    return best_scalar, best_rmse

G, prior_probs = generate_barabasi_albert_graph(num_nodes = 1000,
                                                    num_edges_per_node = 2,
                                                    prob_selected = 0.1)
    
edge_dict = generate_edge_probabilities(G,method="degree_based",low=0, high=1)

ic = optimized_independent_cascade(G, prior_probs, edge_dict, 1000)

true_posterior_tensor = torch.tensor(
    [ic[node] for node in ic],
    dtype=torch.float
).view(-1, 1)

optimal_scalar, min_rmse = binary_search_optimal_scalar(
    G, edge_dict, prior_probs, 10, true_posterior_tensor
)
print(f"Optimal scalar: {optimal_scalar:.4f}, RMSE: {min_rmse:.6f}")

Optimal scalar: 0.9785, RMSE: 0.046203


## Barabasi Albert

In [6]:
num_nodes=1000
num_edges_per_node=5 #2,5,10
prob_selected = 0.1
method="degree_based"
num_graphs=1
num_sim=1000
low=0
high=1/3
percentile=95
T=10 # 5, 10, 15

rmse = np.zeros(num_graphs)
pearson = np.zeros(num_graphs)
spearman = np.zeros(num_graphs)
auc = np.zeros(num_graphs)
time_ic_apporx = np.zeros(num_graphs)
time_mc = np.zeros(num_graphs)

for i in range(num_graphs):
    G, prior_probs = generate_barabasi_albert_graph(num_nodes = num_nodes,
                                                    num_edges_per_node = num_edges_per_node,
                                                    prob_selected = prob_selected)
    
    edge_dict = generate_edge_probabilities(G,method=method,low=low, high=high)

    start = time.time()
    ic = optimized_independent_cascade(G, prior_probs, edge_dict, num_sim)
    end = time.time()
    time_mc[i] = end - start
    true_posterior_tensor = torch.tensor(
        [ic[node] for node in ic],
        dtype=torch.float
    ).view(-1, 1)

    start = time.time()
    preds = IC_approx_vectorized(G, edge_dict, prior_probs, T, 1)
    end = time.time()
    time_ic_apporx[i] = end - start
    
    preds_IC_approx = torch.tensor(preds, dtype=torch.float32).unsqueeze(1)
    rmse[i] = torch.nn.MSELoss()(preds_IC_approx, true_posterior_tensor).item()**0.5

    pearson[i] = torch.corrcoef(torch.stack([preds_IC_approx.squeeze(), true_posterior_tensor.squeeze()]))[0, 1]
    spearman[i], _ = spearmanr(preds_IC_approx.squeeze(),
                              true_posterior_tensor.squeeze())

    y_true = true_posterior_tensor.squeeze().numpy()
    y_score = preds_IC_approx.squeeze().numpy()
    threshold = np.percentile(y_true, percentile)
    y_binary = (y_true >= threshold).astype(int)
    auc[i] = roc_auc_score(y_binary, y_score)



In [7]:
summarise_stats(rmse, pearson, spearman, auc, time_ic_apporx, time_mc)

rmse:
 mean=0.017662977684403402,
 ci_low=nan,
 ci_high=nan,
 std=0.0
pearson:
 mean=0.9948218464851379,
 ci_low=nan,
 ci_high=nan,
 std=0.0
spearman:
 mean=0.9264459246869542,
 ci_low=nan,
 ci_high=nan,
 std=0.0
auc:
 mean=0.9998315789473684,
 ci_low=nan,
 ci_high=nan,
 std=0.0
time_ic_apporx:
 mean=0.006918430328369141,
 ci_low=nan,
 ci_high=nan,
 std=0.0
time_mc:
 mean=1.7893140316009521,
 ci_low=nan,
 ci_high=nan,
 std=0.0


/home/matic/anaconda3/lib/python3.12/site-packages/numpy/core/_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/home/matic/anaconda3/lib/python3.12/site-packages/numpy/core/_methods.py:198: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


## Erdos Renyi

In [28]:
num_nodes=10000
edge_prob=0.002 #0.0004, 0.0010, 0.0020
prob_selected = 0.1
method="degree_based"
num_graphs=10
num_sim=10000
low=0
high=1/3
percentile=95
T=15 # 5, 10, 15

rmse = np.zeros(num_graphs)
pearson = np.zeros(num_graphs)
spearman = np.zeros(num_graphs)
auc = np.zeros(num_graphs)
time_ic_apporx = np.zeros(num_graphs)
time_mc = np.zeros(num_graphs)

for i in range(num_graphs):
    G, prior_probs = generate_erdos_renyi_graph(num_nodes = num_nodes,
                                                    edge_prob = edge_prob,
                                                    prob_selected = prob_selected,
                                                    seed=seed)
    
    edge_dict = generate_edge_probabilities(G,method=method,low=low, high=high)

    start = time.time()
    ic = optimized_independent_cascade(G, prior_probs, edge_dict, num_sim)
    end = time.time()
    time_mc[i] = end - start
    true_posterior_tensor = torch.tensor(
        [ic[node] for node in ic],
        dtype=torch.float
    ).view(-1, 1)

    start = time.time()
    preds = IC_approx_vectorized(G, edge_dict, prior_probs, T, 1)
    end = time.time()
    time_ic_apporx[i] = end - start
    
    preds_IC_approx = torch.tensor(preds, dtype=torch.float32).unsqueeze(1)
    rmse[i] = torch.nn.MSELoss()(preds_IC_approx, true_posterior_tensor).item()**0.5

    pearson[i] = torch.corrcoef(torch.stack([preds_IC_approx.squeeze(), true_posterior_tensor.squeeze()]))[0, 1]
    spearman[i], _ = spearmanr(preds_IC_approx.squeeze(),
                              true_posterior_tensor.squeeze())

    y_true = true_posterior_tensor.squeeze().numpy()
    y_score = preds_IC_approx.squeeze().numpy()
    threshold = np.percentile(y_true, percentile)
    y_binary = (y_true >= threshold).astype(int)
    auc[i] = roc_auc_score(y_binary, y_score)
    
summarise_stats(rmse, pearson, spearman, auc, time_ic_apporx, time_mc)

## Anybeat & Advogato

In [30]:
def load_directed_graph_from_edges(file_path):
    """
    Reads a file containing edge list data and creates a directed graph using NetworkX.

    Each line in the file should contain two integers: source target

    Parameters:
    - file_path (str): Path to the edge list file.

    Returns:
    - G (networkx.DiGraph): Directed graph constructed from the edge list.
    """
    G = nx.DiGraph()  # Directed graph

    with open(file_path, 'r') as f:
        for line in f:
            if line.strip():  # skip empty lines
                source, target = map(int, line.strip().split())
                G.add_edge(source, target)

    return G

def load_weighted_directed_graph(file_path):
    """
    Loads a weighted directed graph from a file.
    Ignores lines starting with '%'. Assumes each valid line has: source target weight

    Returns:
    - G (nx.DiGraph): The directed graph.
    - edge_weight (dict): Dictionary with (u, v) -> weight mapping.
    """
    G = nx.DiGraph()
    edge_weight = {}

    with open(file_path, 'r') as f:
        for line in f:
            line = line.strip()
            if not line or line.startswith('%'):
                continue  # Skip comments and empty lines

            parts = line.split()
            if len(parts) != 3:
                continue  # Skip malformed lines

            u, v = int(parts[0]), int(parts[1])
            w = float(parts[2])

            G.add_edge(u, v, weight=w)
            edge_weight[(u, v)] = w

    return G, edge_weight

In [70]:
prob_selected = 0.1
method="degree_based"
num_sim=10000
low=0
high=1/3
percentile=95
T=20
num_iter = 10

rmse_anybeat = np.zeros(num_iter)
pearson_anybeat = np.zeros(num_iter)
spearman_anybeat = np.zeros(num_iter)
auc_anybeat = np.zeros(num_iter)
time_ic_apporx_anybeat = np.zeros(num_iter)
time_mc_anybeat = np.zeros(num_iter)

G_anybeat = load_directed_graph_from_edges("soc-anybeat/soc-anybeat.edges")
edge_dict_anybeat = generate_edge_probabilities(G_anybeat,method=method,low=low, high=high)
for i in range(num_iter):
    
    mask = torch.rand(len(G_anybeat.nodes())) < prob_selected
    prior_probs_anybeat = mask.float()  * torch.rand(len(G_anybeat.nodes()))
    
    start = time.time()
    ic_anybeat = optimized_independent_cascade(G_anybeat, prior_probs_anybeat, edge_dict_anybeat, num_sim)
    end = time.time()
    time_mc_anybeat[i] = end - start
    true_posterior_tensor_anybeat = torch.tensor(
        [ic_anybeat[node] for node in ic_anybeat],
        dtype=torch.float
    ).view(-1, 1)
    
    start = time.time()
    preds_anybeat = IC_approx_vectorized(G_anybeat, edge_dict_anybeat, prior_probs_anybeat, T, 1)
    end = time.time()
    time_ic_apporx_anybeat[i] = end - start
    
    preds_IC_approx_anybeat = torch.tensor(preds_anybeat, dtype=torch.float32).unsqueeze(1)
    rmse_anybeat[i] = torch.nn.MSELoss()(preds_IC_approx_anybeat, true_posterior_tensor_anybeat).item()**0.5
    
    pearson_anybeat[i] = torch.corrcoef(torch.stack([preds_IC_approx_anybeat.squeeze(), true_posterior_tensor_anybeat.squeeze()]))[0, 1]
    spearman_anybeat[i], _ = spearmanr(preds_IC_approx_anybeat.squeeze(),
                              true_posterior_tensor_anybeat.squeeze())
    
    y_true = true_posterior_tensor_anybeat.squeeze().numpy()
    y_score = preds_IC_approx_anybeat.squeeze().numpy()
    threshold = np.percentile(y_true, percentile)
    y_binary = (y_true >= threshold).astype(int)
    auc_anybeat[i] = roc_auc_score(y_binary, y_score)

summarise_stats(rmse_anybeat, pearson_anybeat, spearman_anybeat, auc_anybeat, time_ic_apporx_anybeat, time_mc_anybeat)

rmse:
 mean=0.06650379837585291,
 ci_low=0.05946359044118863,
 ci_high=0.07354400631051719,
 std=0.009336497105862238
pearson:
 mean=0.9697460353374481,
 ci_low=0.966707976616785,
 ci_high=0.9727840940581112,
 std=0.004028975666080733
spearman:
 mean=0.9671038871004273,
 ci_low=0.9590673325514546,
 ci_high=0.9751404416494,
 std=0.010657819908719066
auc:
 mean=0.9924521677055849,
 ci_low=0.9898154165407712,
 ci_high=0.9950889188703985,
 std=0.0034967745054727663
time_ic_apporx:
 mean=0.06618521213531495,
 ci_low=0.05633780439727954,
 ci_high=0.07603261987335036,
 std=0.013059315108254302
time_mc:
 mean=150.1168033361435,
 ci_low=131.10356861711094,
 ci_high=169.13003805517607,
 std=25.214739759784127


In [71]:
prob_selected = 0.1
method="degree_based"
num_sim=10000
low=0
high=1/3
percentile=95
T=5
num_iter = 10

rmse_advogato = np.zeros(num_iter)
pearson_advogato = np.zeros(num_iter)
spearman_advogato = np.zeros(num_iter)
auc_advogato = np.zeros(num_iter)
time_ic_apporx_advogato = np.zeros(num_iter)
time_mc_advogato = np.zeros(num_iter)

G_advogato, weights_advogato = load_weighted_directed_graph("soc-advogato/soc-advogato.edges")
edge_dict_advogato = generate_edge_probabilities(G_advogato,method=method,low=low, high=high)

for edge in edge_dict_advogato:
    edge_dict_advogato[edge] *= weights_advogato[edge]
    
for i in range(num_iter):
    mask = torch.rand(len(G_advogato.nodes())) < prob_selected
    prior_probs_advogato = mask.float()  * torch.rand(len(G_advogato.nodes()))
    
    start = time.time()
    ic_advogato = optimized_independent_cascade(G_advogato, prior_probs_advogato, edge_dict_advogato, num_sim)
    end = time.time()
    time_mc_advogato[i] = end - start
    true_posterior_tensor_advogato = torch.tensor(
        [ic_advogato[node] for node in ic_advogato],
        dtype=torch.float
    ).view(-1, 1)
    
    start = time.time()
    preds_advogato = IC_approx_vectorized(G_advogato, edge_dict_advogato, prior_probs_advogato, T, 1)
    end = time.time()
    time_ic_apporx_advogato[i] = end - start
    
    preds_IC_approx_advogato = torch.tensor(preds_advogato, dtype=torch.float32).unsqueeze(1)
    rmse_advogato[i] = torch.nn.MSELoss()(preds_IC_approx_advogato, true_posterior_tensor_advogato).item()**0.5
    
    pearson_advogato[i] = torch.corrcoef(torch.stack([preds_IC_approx_advogato.squeeze(), true_posterior_tensor_advogato.squeeze()]))[0, 1]
    spearman_advogato[i], _ = spearmanr(preds_IC_approx_advogato.squeeze(),
                              true_posterior_tensor_advogato.squeeze())
    
    y_true = true_posterior_tensor_advogato.squeeze().numpy()
    y_score = preds_IC_approx_advogato.squeeze().numpy()
    threshold = np.percentile(y_true, percentile)
    y_binary = (y_true >= threshold).astype(int)
    auc_advogato[i] = roc_auc_score(y_binary, y_score)

summarise_stats(rmse_advogato, pearson_advogato, spearman_advogato, auc_advogato, time_ic_apporx_advogato, time_mc_advogato)

rmse:
 mean=0.037563673690395885,
 ci_low=0.03577886026847289,
 ci_high=0.03934848711231888,
 std=0.0023669620986958412
pearson:
 mean=0.9863605558872223,
 ci_low=0.9859092629794423,
 ci_high=0.9868118487950023,
 std=0.0005984901250767988
spearman:
 mean=0.9967038268130173,
 ci_low=0.9965585768402414,
 ci_high=0.9968490767857932,
 std=0.00019262583762209617
auc:
 mean=0.997084649588662,
 ci_low=0.9961340172387791,
 ci_high=0.998035281938545,
 std=0.0012606980171309234
time_ic_apporx:
 mean=0.047872185707092285,
 ci_low=0.038153697425194755,
 ci_high=0.057590673988989816,
 std=0.012888346276042099
time_mc:
 mean=50.73339323997497,
 ci_low=45.170897128664244,
 ci_high=56.2958893512857,
 std=7.376803260158133


In [79]:
num_nodes=10000
num_edges_per_node=5 #2,5,10
prob_selected = 0.1
edge_method="weighted_sum"
num_graphs=1
num_sim=1000
low=0
high=1/3
percentile=95
T=15 # 5, 10, 15
num_node_features=5
num_edge_features=5
attribute_distribution="normal"
max_edge=1
normal = False
if normal:
    node_weights=np.random.normal(size=num_node_features)
    node_weights_source=np.random.normal(size=num_node_features)
    node_weights_sink=np.random.normal(size=num_node_features)
    epsilon=np.random.normal(size=1)*-4
    edge_weights=np.random.normal(size=num_edge_features)
else:
    node_weights=np.random.rand(num_node_features)*2-1
    node_weights_source=np.random.rand(num_node_features)*2-1
    node_weights_sink=np.random.rand(num_node_features)*2-1
    epsilon=np.random.rand(1)*-4
    edge_weights=np.random.rand(num_edge_features)*2-1

rmse = np.zeros(num_graphs)
pearson = np.zeros(num_graphs)
spearman = np.zeros(num_graphs)
auc = np.zeros(num_graphs)
time_ic_apporx = np.zeros(num_graphs)
time_mc = np.zeros(num_graphs)

for i in range(num_graphs):
    G, prior_probs, edge_dict = generate_barabasi_albert_attribute_graph(num_nodes,
                                                                    num_edges_per_node,
                                                                    num_node_features=num_node_features,
                                                                    num_edge_features=num_edge_features,
                                                                    node_weights=node_weights,
                                                                    edge_weights=edge_weights,
                                                                    edge_method=edge_method,
                                                                    max_edge=max_edge,
                                                                    prob_selected=prob_selected,
                                                                    attribute_distribution=attribute_distribution)

    edge_dict_deg = generate_edge_probabilities(G,method="degree_based",low=low, high=high)

    for edge in edge_dict:
        edge_dict[edge] *= edge_dict_deg[edge]
    
    start = time.time()
    ic = optimized_independent_cascade(G, prior_probs, edge_dict, num_sim)
    end = time.time()
    time_mc[i] = end - start
    true_posterior_tensor = torch.tensor(
        [ic[node] for node in ic],
        dtype=torch.float
    ).view(-1, 1)

    start = time.time()
    preds = IC_approx_vectorized(G, edge_dict, prior_probs, T, 1)
    end = time.time()
    time_ic_apporx[i] = end - start
    
    preds_IC_approx = torch.tensor(preds, dtype=torch.float32).unsqueeze(1)
    rmse[i] = torch.nn.MSELoss()(preds_IC_approx, true_posterior_tensor).item()**0.5

    pearson[i] = torch.corrcoef(torch.stack([preds_IC_approx.squeeze(), true_posterior_tensor.squeeze()]))[0, 1]
    spearman[i], _ = spearmanr(preds_IC_approx.squeeze(),
                              true_posterior_tensor.squeeze())

    y_true = true_posterior_tensor.squeeze().numpy()
    y_score = preds_IC_approx.squeeze().numpy()
    threshold = np.percentile(y_true, percentile)
    y_binary = (y_true >= threshold).astype(int)
    auc[i] = roc_auc_score(y_binary, y_score)

summarise_stats(rmse, pearson, spearman, auc, time_ic_apporx, time_mc)

rmse:
 mean=0.008081818563251842,
 ci_low=nan,
 ci_high=nan,
 std=0.0
pearson:
 mean=0.9989129304885864,
 ci_low=nan,
 ci_high=nan,
 std=0.0
spearman:
 mean=0.9706517710905714,
 ci_low=nan,
 ci_high=nan,
 std=0.0
auc:
 mean=0.9999418947368421,
 ci_low=nan,
 ci_high=nan,
 std=0.0
time_ic_apporx:
 mean=0.058057546615600586,
 ci_low=nan,
 ci_high=nan,
 std=0.0
time_mc:
 mean=5.283751964569092,
 ci_low=nan,
 ci_high=nan,
 std=0.0


/home/matic/anaconda3/lib/python3.12/site-packages/numpy/core/_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/home/matic/anaconda3/lib/python3.12/site-packages/numpy/core/_methods.py:198: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
